In [ ]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.cbis_ddsm import CBISDDSMDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score

from catboost import CatBoostClassifier

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [ ]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/cbis_ddsm")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/cbis_ddsm")

kind = 'calc'  # mass calc
image_type = 'all' # all full crop roi
test_dataset = CBISDDSMDataset(data_path=data_path, data_name=f'csv/{kind}_case_description_test_set.csv', kind=kind, image_type=image_type)
# _ = test_dataset.get_images()
# _ = test_dataset.get_embeddings(mode='test')
train_dataset = CBISDDSMDataset(data_path=data_path, data_name=f'csv/{kind}_case_description_train_set.csv', kind=kind, image_type=image_type)
# _ = train_dataset.get_images()
# _ = train_dataset.get_embeddings(mode='train')

In [3]:
accuracy_scores = []
for seed in range(10):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    y_train = pd.Series(y_train)
    y_test = pd.Series(y_test)
    
    X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
    X_test.iloc[:, 0:19] = X_test.iloc[:, 0:19].astype('str')
    
    model = CatBoostClassifier(cat_features=list(range(18)))

    # Train the model
    model.fit(X_train, y_train, verbose=False)

    acc_score = accuracy_score(y_test, model.predict(X_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.0
1       -1.0
2       -1.0
3        1.0
4       -1.0
        ... 
1833     0.0
1834     0.0
1835    -1.0
1836    -1.0
1837     1.0
Name: 0, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.0
1       -1.0
2       -1.0
3        1.0
4       -1.0
        ... 
1833     0.0
1834     0.0
1835    -1.0
1836    -1.0
1837     1.0
Name: 1, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2

accuracy_score (Finetuned): 0.8021739130434783


/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.0
1        0.0
2        0.0
3        1.0
4       -1.0
        ... 
1833    -1.0
1834     0.0
1835     1.0
1836    -1.0
1837     0.0
Name: 0, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.0
1        1.0
2        0.0
3        1.0
4       -1.0
        ... 
1833    -1.0
1834     0.0
1835     1.0
1836    -1.0
1837     0.0
Name: 1, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2

accuracy_score (Finetuned): 0.7673913043478261


/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.0
1        0.0
2        0.0
3        1.0
4       -1.0
        ... 
1833     0.0
1834     0.0
1835     0.0
1836    -1.0
1837     1.0
Name: 0, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.0
1        1.0
2        0.0
3        0.0
4       -1.0
        ... 
1833     1.0
1834     0.0
1835     0.0
1836    -1.0
1837     1.0
Name: 1, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2

...

accuracy_score (Finetuned): 0.7717391304347826


/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
1833    -1.0
1834     0.0
1835     0.0
1836     0.0
1837    -1.0
Name: 0, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        1.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
1833    -1.0
1834     1.0
1835     1.0
1836     1.0
1837    -1.0
Name: 1, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2

accuracy_score (Finetuned): 0.7760869565217391


/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        1.0
1        0.0
2       -1.0
3        0.0
4        0.0
        ... 
1833     0.0
1834    -1.0
1835     0.0
1836     0.0
1837     0.0
Name: 0, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2982852803.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        1.0
1        0.0
2       -1.0
3        0.0
4        0.0
        ... 
1833     1.0
1834    -1.0
1835     0.0
1836     0.0
1837     0.0
Name: 1, Length: 1838, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 0:19] = X_train.iloc[:, 0:19].astype('str')
/tmp/ipykernel_16266/2

accuracy_score (Finetuned): 0.8043478260869565


In [4]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.788913043478261
Std Accuracy: 0.018043478260869574
